In [1]:
## QICK LIBRARY
from qick import *

from random import random
import time
import pickle
import numpy as np

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger

##### Load FGPA BitStream

##################
soc = QickSoc('./qick_111_rfbv2.bit')


In [3]:
soc.tproc.info()

---------------------------------------------
 TPROC V2 INFO 
---------------------------------------------
pmem_size     : 1024
dmem_size     : 1024
wmem_size     : 1024
dreg_qty      : 16
in_port_qty   : 2
out_trig_qty  : 2
out_dport_qty : 2
out_dport_dw  : 8
out_wport_qty : 8

Configuration:
has_io_ctrl   : YES

Peripherals:
has_lfsr      : NO
has_divider   : NO
has_arith     : YES
has_time_read : YES
has_tnet      : NO
has_custom_periph: NO


In [2]:
asm = """
//TEST program
// Write CORE_W_DT SFR
REG_WR s12 imm #12
REG_WR s13 imm #13

// Calculate 9 / 4 > Q=2 R=1
REG_WR r1 imm #9
REG_WR r2 imm #4
DIV r1 r2
// Calculate (2+3)*5-10 = 15
REG_WR r1 imm #2
REG_WR r2 imm #3
REG_WR r3 imm #5
REG_WR r4 imm #10
ARITH PTM r1 r2 r3 r4
FLAG set
"""
p_txt, p_bin  = Assembler.str_asm2bin(asm)

soc.tproc.Load_PMEM(p_bin)

soc.tproc.start()
soc.tproc.stop()

soc.tproc.tproc_w_dt1 = 12345
soc.tproc.tproc_w_dt2 = 6789
soc.tproc.core_cfg  = 1

src_list = ['TPROC_W_DT','CORE0_W_DT','RFU','DIV (Q-R)','ARITH(L-H)','TNET','PERIPH','PORT','RAND','RFU','RFU','RFU','RFU','RFU','RFU','RFU',]
for ind in range(16):
    soc.tproc.read_sel  = ind
    print("{:10} ( READ_SEL={:2} ) = {:10}-{:10}".format(src_list[ind],soc.tproc.read_sel, soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2) )



TPROC_W_DT ( READ_SEL= 0 ) =      12345-      6789
CORE0_W_DT ( READ_SEL= 1 ) =         12-        13
RFU        ( READ_SEL= 2 ) =          0-         0
DIV (Q-R)  ( READ_SEL= 3 ) =          0-         0
ARITH(L-H) ( READ_SEL= 4 ) =         15-         0
TNET       ( READ_SEL= 5 ) =          0-         0
PERIPH     ( READ_SEL= 6 ) =          0-         0
PORT       ( READ_SEL= 7 ) =          0-         0
RAND       ( READ_SEL= 8 ) =          0-         0
RFU        ( READ_SEL= 9 ) =          0-         0
RFU        ( READ_SEL=10 ) =          0-         0
RFU        ( READ_SEL=11 ) =          0-         0
RFU        ( READ_SEL=12 ) =          0-         0
RFU        ( READ_SEL=13 ) =          0-         0
RFU        ( READ_SEL=14 ) =          0-         0
RFU        ( READ_SEL=15 ) =          0-         0


In [ ]:
soc.tproc.start()
soc.tproc.get_axi()
soc.tproc.get_status()
soc.tproc.get_debug()
print('TIME_USR: ', soc.tproc.time_usr)
print('TIME_USR: ', soc.tproc.time_usr)



In [40]:
asm = """
REG_WR s12 imm #0 
REG_WR s13 imm #100

TIME #384 inc_ref 

REG_WR r0 imm #10

REPS:
     REG_WR s14 imm #0 
     // Send Waveform
     WPORT_WR p7 wmem [&0] 
     // Send Pulse
     REG_WR s14 imm #100 
     DPORT_WR p1 imm 1 
     DPORT_WR p0 imm 1 
     REG_WR s14 imm #200
     DPORT_WR p1 imm 0 
     DPORT_WR p0 imm 0 
     
     WAIT [&13] @500
     DMEM_WR [s12] op -op(s10) 
     DPORT_RD p0 
     //DMEM_WR [s12] op -op(s8) 
     
     WAIT [&17] @153 
     TIME #38553 inc_ref 
     REG_WR s12 op -op(s12 + #1) 
     REG_WR r0 op -op(r0-#1) -uf 
     REG_WR s13 op -op(s13 + #1) 
     JUMP REPS -if(NZ)
     REG_WR s13 op -op(s13 + #1000) 
     JUMP HERE 
"""
p_txt, p_bin  = Assembler.str_asm2bin(asm)

soc.tproc.Load_PMEM(p_bin)

soc.tproc.start()


soc.tproc.tproc_w_dt1 = 12345
soc.tproc.tproc_w_dt2 = 6789
soc.tproc.core_cfg  = 1

src_list = ['TPROC_W_DT','CORE0_W_DT','RFU','DIV (Q-R)','ARITH(L-H)','TNET','PERIPH','PORT','RAND','RFU','RFU','RFU','RFU','RFU','RFU','RFU',]
for ind in range(16):
    soc.tproc.read_sel  = ind
    print("{:10} ( READ_SEL={:2} ) = {:10}-{:10}".format(src_list[ind],soc.tproc.read_sel, soc.tproc.tproc_r_dt1, soc.tproc.tproc_r_dt2) )


print('---------------------------------------------')
print("Reading Data Memory:")
Data_qty = 10
#read_mem(mem_sel, addr, length):
rd_buff = soc.tproc.read_mem(2, 0, Data_qty)
print(rd_buff)         

TPROC_W_DT ( READ_SEL= 0 ) =      12345-      6789
CORE0_W_DT ( READ_SEL= 1 ) =         10-      1110
RFU        ( READ_SEL= 2 ) =          0-         0
DIV (Q-R)  ( READ_SEL= 3 ) =          0-         0
ARITH(L-H) ( READ_SEL= 4 ) =         15-         0
TNET       ( READ_SEL= 5 ) =          0-         0
PERIPH     ( READ_SEL= 6 ) =          0-         0
PORT       ( READ_SEL= 7 ) =          0-         0
RAND       ( READ_SEL= 8 ) =          0-         0
RFU        ( READ_SEL= 9 ) =          0-         0
RFU        ( READ_SEL=10 ) =          0-         0
RFU        ( READ_SEL=11 ) =          0-         0
RFU        ( READ_SEL=12 ) =          0-         0
RFU        ( READ_SEL=13 ) =          0-         0
RFU        ( READ_SEL=14 ) =          0-         0
RFU        ( READ_SEL=15 ) =          0-         0
---------------------------------------------
Reading Data Memory:
[[1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1